## Adding missing coordinates

In [1]:
import pandas as pd
import requests
import time
from tqdm import tqdm

def geocode_with_nominatim(block, street_name):
    url = "https://nominatim.openstreetmap.org/search"
    
    address_formats = [
        f"{block} {street_name}, Singapore",
        f"Block {block} {street_name}, Singapore",
        f"{block} {street_name}, SG",
    ]
    
    headers = {'User-Agent': 'HDB-Geocoder/1.0'}
    
    for address in address_formats:
        params = {
            'q': address,
            'format': 'json',
            'limit': 1,
            'countrycodes': 'sg'
        }
        
        try:
            response = requests.get(url, params=params, headers=headers, timeout=10)
            data = response.json()
            
            if len(data) > 0:
                result = data[0]
                lat = float(result['lat'])
                lon = float(result['lon'])
                
                if 1.15 <= lat <= 1.47 and 103.6 <= lon <= 104.0:
                    return (lat, lon)
            
            time.sleep(1.1)
        except:
            time.sleep(1.1)
    
    return (None, None)

# Load dataset
df = pd.read_csv("C:/Users/CJSteve/Desktop/HDB2/hdb_with_coordinates.csv")
print(f"Loaded {len(df):,} rows")

# Expand all abbreviations
df['street_name'] = df['street_name'].str.replace("C'WEALTH", "COMMONWEALTH", regex=False)
df['street_name'] = df['street_name'].str.replace("ST.", "SAINT", regex=False)
df['street_name'] = df['street_name'].str.replace("KG ", "KAMPONG ", regex=False)
df['street_name'] = df['street_name'].str.replace(" PK ", " PARK ", regex=False)
df['street_name'] = df['street_name'].str.replace(" MKT ", " MARKET ", regex=False)
df['street_name'] = df['street_name'].str.replace(" RD", " ROAD", regex=False)
df['street_name'] = df['street_name'].str.replace(" DR", " DRIVE", regex=False)
df['street_name'] = df['street_name'].str.replace(" CL", " CLOSE", regex=False)
df['street_name'] = df['street_name'].str.replace(" AVE", " AVENUE", regex=False)
df['street_name'] = df['street_name'].str.replace(" CRES", " CRESCENT", regex=False)

# Find missing
missing_mask = df['latitude'].isna() | df['longitude'].isna()
missing_df = df[missing_mask]
unique_missing = missing_df[['block', 'street_name']].drop_duplicates()

print(f"Geocoding {len(unique_missing):,} unique addresses...")

# Geocode
address_coords = {}
for idx, row in tqdm(unique_missing.iterrows(), total=len(unique_missing)):
    key = (str(row['block']), str(row['street_name']))
    lat, lon = geocode_with_nominatim(row['block'], row['street_name'])
    address_coords[key] = (lat, lon)

# Map back
for idx, row in missing_df.iterrows():
    key = (str(row['block']), str(row['street_name']))
    if key in address_coords:
        lat, lon = address_coords[key]
        df.at[idx, 'latitude'] = lat
        df.at[idx, 'longitude'] = lon

# Save
df.to_csv("Complete_HDB_resale_dataset_2015_to_2016.csv", index=False)
print(f"Done! Missing: {df['latitude'].isna().sum()}")

Loaded 37,153 rows
Geocoding 74 unique addresses...


100%|██████████████████████████████████████████████████████████████████████████████████| 74/74 [01:14<00:00,  1.01s/it]

Done! Missing: 0


## Mapping region to Towns

In [2]:
import pandas as pd

# Define region mappings (town names in uppercase)
region_mapping = {
    'CCR': [
        'BISHAN', 'BUKIT MERAH', 'BUKIT TIMAH', 'CENTRAL AREA', 
        'GEYLANG', 'KALLANG/WHAMPOA', 'MARINE PARADE', 
        'QUEENSTOWN', 'TOA PAYOH'
    ],
    'RCR': [
        'ANG MO KIO', 'BEDOK', 'BUKIT PANJANG', 'CLEMENTI', 
        'HOUGANG', 'PASIR RIS', 'PUNGGOL', 'SERANGOON', 
        'SENGKANG', 'TAMPINES'
    ],
    'OCR': [
        'BUKIT BATOK', 'CHOA CHU KANG', 'JURONG EAST', 
        'JURONG WEST', 'SEMBAWANG', 'WOODLANDS', 'YISHUN'
    ]
}

# Create a flat mapping dictionary
town_to_region = {}
for region, towns in region_mapping.items():
    for town in towns:
        town_to_region[town] = region

# Read your HDB dataset
input_file = '03_dataset_for_ML_model/Complete_HDB_resale_dataset_2015_to_2016.csv'  
df = pd.read_csv(input_file)

print(f"Original shape: {df.shape}")

# Map towns to regions
df['region'] = df['town'].map(town_to_region)

# Check for unmapped towns
unmapped = df[df['region'].isna()]['town'].unique()
if len(unmapped) > 0:
    print(f"WARNING: Unmapped towns: {unmapped}")

# Get the position of 'town' column and place 'region' before it
town_col_idx = df.columns.get_loc('town')
cols = df.columns.tolist()
cols.remove('region')
cols.insert(town_col_idx, 'region')
df = df[cols]

# Display region distribution
print("\nRegion distribution:")
print(df['region'].value_counts().sort_index())

# Save back to the same file (overwrites existing)
df.to_csv(input_file, index=False)

print(f"\n✓ Region column added successfully to {input_file}")
print(f"Sample:\n{df[['region', 'town']].head(10)}")

Original shape: (37153, 13)

Region distribution:
region
CCR     7016
OCR    12758
RCR    17379
Name: count, dtype: int64

✓ Region column added successfully to 03_dataset_for_ML_model/Complete_HDB_resale_dataset_2015_to_2016.csv
Sample:
  region        town
0    RCR  ANG MO KIO
1    RCR  ANG MO KIO
2    RCR  ANG MO KIO
3    RCR  ANG MO KIO
4    RCR  ANG MO KIO
5    RCR  ANG MO KIO
6    RCR  ANG MO KIO
7    RCR  ANG MO KIO
8    RCR  ANG MO KIO
9    RCR  ANG MO KIO
